In [1]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import tensorflow as tf
from baselines.ppo1 import cnn_policy_particle_sweep_mult_density
import baselines.common.tf_util as U
import gym
import pandas as pd
%matplotlib inline

# Load Saved Trajectory

In [2]:
traj_data = joblib.load("data/trajs/plasticOneClusterBarCentric.pkl")
observations = traj_data["observations"]
observations = np.array(observations)
actions = traj_data["actions"]
actions = np.array(actions)

print("Observation Shape: ",observations.shape)
print("Action Shape: ", actions.shape)

#print("Split observations into separate scenes")
obs_at_frame = observations[0,0]


Observation Shape:  (1000, 49, 2052)
Action Shape:  (1000, 49, 4)


# Load Saved Policy

In [36]:
def restore_policy(sess, policy, policy_params):
    cur_scope = policy.get_variables()[0].name[0:policy.get_variables()[0].name.find('/')]
    orig_scope = list(policy_params.keys())[0][0:list(policy_params.keys())[0].find('/')]

    for i in range(len(policy.get_variables())):
        assign_op = policy.get_variables()[i].assign(
            policy_params[policy.get_variables()[i].name.replace(cur_scope, orig_scope, 1)])
        sess.run(assign_op)

def cnn_policy_fn(name, ob_space, ac_space):  # pylint: disable=W0613
    return cnn_policy_particle_sweep_mult_density.CnnPolicyParticleSweepMultiDensity(name=name, ob_space=ob_space,
                                                                             ac_space=ac_space
                                                                             )

policy_filename = "data/trajs/policies/"
policy_filename +="singleClusterFixed/policy_params_5.pkl"
# print(policy_filename)
env = gym.make("FlexPlasticReshaping-v6")
policy_param =joblib.load(policy_filename)

tf.reset_default_graph()
activation1 = None
activation2 = None

saliencies = None
with U.single_threaded_session() as sess:
    pi = cnn_policy_fn('pi', env.observation_space, env.action_space)
    restore_policy(sess, pi, policy_param)

#     tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='pi/pol/conv_1')
    # curr_particle_density_diff = obs_at_frame[15:15+32*32]
    # curr_particle_density_diff = np.reshape(curr_particle_density_diff,(32,32))
    # prev_particle_density_diff = obs_at_frame[15+32*32::]
    # prev_particle_density_diff = np.reshape(prev_particle_density_diff,(32,32))
    # conv_obs = np.concatenate([curr_particle_density_diff,prev_particle_density_diff],axis=-1)
#     for i in range(1000):
    saliencies = pi.get_saliency(observations[:,0])[0][0]
    
print(saliencies.shape)


[2019-10-31 17:28:09,398] Making new env: FlexPlasticReshaping-v6


pyFlex initialization OK
image 0 Conv input Shape:  (?, 32, 32, 1)
image 0 Conv1 out Shape:  (?, 16, 16, 8)
image 0 Conv2 out Shape:  (?, 8, 8, 4)
image 0 Conv3 out Shape:  (?, 8, 8, 2)
image 1 Conv input Shape:  (?, 32, 32, 1)
image 1 Conv1 out Shape:  (?, 16, 16, 8)
image 1 Conv2 out Shape:  (?, 8, 8, 4)
image 1 Conv3 out Shape:  (?, 8, 8, 2)
First FC Shape:  (?, 260)
image 0 Conv input Shape:  (?, 32, 32, 1)
image 0 Conv1 out Shape:  (?, 16, 16, 8)
image 0 Conv2 out Shape:  (?, 8, 8, 4)
image 0 Conv3 out Shape:  (?, 8, 8, 2)
image 1 Conv input Shape:  (?, 32, 32, 1)
image 1 Conv1 out Shape:  (?, 16, 16, 8)
image 1 Conv2 out Shape:  (?, 8, 8, 4)
image 1 Conv3 out Shape:  (?, 8, 8, 2)
First FC Shape:  (?, 260)
(1000, 2052)


# Draw Saliency Maps

In [37]:

def saliency_at_frame(i):
    saliency = saliencies[i]
    obs = observations[i,0]
    density_saliency = saliency[4:4+32*32]
    density_saliency = np.reshape(density_saliency,(32,32))
    density_saliency-=np.min(density_saliency)
    goal_saliency = saliency[4+32*32:4+2*32*32]
    goal_saliency = np.reshape(goal_saliency,(32,32))

    bar_vel_saliency = saliency[:4]
    bar_vel_saliency = np.expand_dims(bar_vel_saliency,axis=0)
    
    density_obs = obs[4:4+32*32]
    density_obs = np.reshape(density_obs,(32,32))
    
    goal_obs = obs[4+32*32:4+2*32*32]
    goal_obs= np.reshape(goal_obs,(32,32))
    
    bar_vel_obs = obs[:4]
    bar_vel_obs = np.expand_dims(bar_vel_obs,axis=0)

    fig, axes = plt.subplots(2,3,figsize=(10,10))
    axes[0,0].imshow(density_saliency, vmin=0)
    axes[0,1].imshow(goal_saliency,  vmin=0)
    img1 = axes[0,2].imshow(bar_vel_saliency,  vmin=0)

    axes[1,0].imshow(density_obs,  vmin=0)
    axes[1,1].imshow(goal_obs,  vmin=0)
    img2 = axes[1,2].imshow(bar_vel_obs,  vmin=0)
    
    fig.tight_layout()
    
#     print (pd.DataFrame(density_saliency))
w = widgets.IntSlider(min=0,max=999)

# plot_at_frame(0,fig,axes)
interact(saliency_at_frame, i = w)

interactive(children=(IntSlider(value=0, description='i', max=999), Output()), _dom_classes=('widget-interact'…

<function __main__.saliency_at_frame(i)>